In [7]:
import os



os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from datetime import datetime

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import load_model
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.metrics import precision_score, recall_score, f1_score


In [8]:
print('-----------------------------------------------------------------------------------------------------------------------------------------------------------')
print("Flow Training ...")

flow_dataset = pd.read_csv('FlowStatsfile.csv')
flow_dataset = flow_dataset.sample(frac=1).reset_index(drop=True)
flow_dataset.iloc[:, 2] = flow_dataset.iloc[:, 2].str.replace('.', '')
flow_dataset.iloc[:, 3] = flow_dataset.iloc[:, 3].str.replace('.', '')
flow_dataset.iloc[:, 5] = flow_dataset.iloc[:, 5].str.replace('.', '')

X_flow = flow_dataset.iloc[:, :-1].values
X_flow = StandardScaler().fit_transform(X_flow)
y_flow = flow_dataset.iloc[:, -1].values

# Undersample the majority class (label 1) before splitting
count_class_0, count_class_1 = np.bincount(y_flow)
indices_class_0 = np.where(y_flow == 0)[0]
indices_class_1 = np.where(y_flow == 1)[0][:count_class_0]
indices_undersampled = np.concatenate([indices_class_0, indices_class_1])
X_flow_undersampled, y_flow_undersampled = X_flow[indices_undersampled], y_flow[indices_undersampled]

# Shuffle the undersampled data
X_flow_undersampled, y_flow_undersampled = shuffle(X_flow_undersampled, y_flow_undersampled)

X_flow_train, X_flow_test, y_flow_train, y_flow_test = train_test_split(X_flow_undersampled, y_flow_undersampled, test_size=0.10, random_state=0)
model = Sequential([

    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.06), input_shape=(X_flow_train.shape[1],)),Dropout(0.6),#Increase neurons in the first layer
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.06)),Dropout(0.65),  # Increase neurons in the second layer
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.06)),Dropout(0.65),  # Increase neurons in the third layer
    Dense(1, activation='sigmoid')  # Increase neurons in the last layer to 1
])
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(X_flow_train, y_flow_train, epochs=5, batch_size=128, validation_data=(X_flow_test, y_flow_test))
# Saving the model as a file in the current directory
model.save('./FCNN_TrainedModel.h5')
flow_model = model
scaler = StandardScaler()
X_flow_train = scaler.fit_transform(X_flow_train)
X_flow_test = scaler.transform(X_flow_test)

y_flow_pre = model.predict(X_flow_test)
y_flow_pred = (y_flow_pre > 0.5).astype(int)
print("------------------------------------------------------------------------------")
print("confusion matrix")
cm = confusion_matrix(y_flow_test, y_flow_pred)
precision = precision_score(y_flow_test, y_flow_pred)
recall = recall_score(y_flow_test, y_flow_pred)
f1 = f1_score(y_flow_test, y_flow_pred)
print(cm)
acc = accuracy_score(y_flow_test, y_flow_pred)
print("success accuracy = {0:.2f} %".format(acc*100))
fail = 1.0 - acc
print("fail accuracy = {0:.2f} %".format(fail*100))
print("Precision: {0:.2f}%".format(precision))
print("Recall: {0:.2f}%".format(recall))
print("F1 Score: {0:.2f}%".format(f1))
print("------------------------------------------------------------------------------")


-----------------------------------------------------------------------------------------------------------------------------------------------------------
Flow Training ...


/tmp/ipykernel_47421/3968409797.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  flow_dataset.iloc[:, 2] = flow_dataset.iloc[:, 2].str.replace('.', '')
/tmp/ipykernel_47421/3968409797.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  flow_dataset.iloc[:, 3] = flow_dataset.iloc[:, 3].str.replace('.', '')
/tmp/ipykernel_47421/3968409797.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  flow_dataset.iloc[:, 5] = flow_dataset.iloc[:, 5].str.replace('.', '')


Epoch 1/5
12813/12813 [==============================] - 43s 3ms/step - loss: 0.3416 - accuracy: 0.9487 - val_loss: 0.1936 - val_accuracy: 0.9627
Epoch 2/5
12813/12813 [==============================] - 42s 3ms/step - loss: 0.2077 - accuracy: 0.9545 - val_loss: 0.1697 - val_accuracy: 0.9643
Epoch 3/5
12813/12813 [==============================] - 41s 3ms/step - loss: 0.1970 - accuracy: 0.9560 - val_loss: 0.1706 - val_accuracy: 0.9663
Epoch 4/5
12813/12813 [==============================] - 43s 3ms/step - loss: 0.1923 - accuracy: 0.9567 - val_loss: 0.1629 - val_accuracy: 0.9670
Epoch 5/5
12813/12813 [==============================] - 42s 3ms/step - loss: 0.1886 - accuracy: 0.9573 - val_loss: 0.1584 - val_accuracy: 0.9695


/home/m3gan/.local/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5695/5695 [==============================] - 6s 1ms/step
------------------------------------------------------------------------------
confusion matrix
[[81928  8833]
 [    3 91460]]
success accuracy = 95.15 %
fail accuracy = 4.85 %
Precision: 0.91%
Recall: 1.00%
F1 Score: 0.95%
------------------------------------------------------------------------------
